In [ ]:
import simpy
import random
import numpy as np

In [ ]:
# Define system parameters
ARRIVAL_RATE = 10  # vehicles per minute (lambda)
SERVICE_RATE = 15  # vehicles per minute (mu)
NUM_LANES = 2  # Number of servers (traffic lanes)
SIM_TIME = 120  # Simulation time in minutes

In [ ]:
class TrafficIntersection:
    def __init__(self, env, num_lanes, service_rate):
        self.env = env
        self.lanes = simpy.Resource(env, num_lanes)
        self.service_rate = service_rate
    
    def serve_vehicle(self):
        service_time = random.expovariate(self.service_rate)
        yield self.env.timeout(service_time)

In [ ]:
# Vehicle arrival process
def vehicle_generator(env, intersection, arrival_rate):
    while True:
        yield env.timeout(random.expovariate(arrival_rate))
        env.process(vehicle_process(env, intersection))

In [ ]:
# Vehicle queuing process
def vehicle_process(env, intersection):
    arrival_time = env.now
    with intersection.lanes.request() as request:
        yield request
        wait_time = env.now - arrival_time
        WAIT_TIMES.append(wait_time)
        yield env.process(intersection.serve_vehicle())

In [ ]:
# Initialize simulation environment
env = simpy.Environment()
intersection = TrafficIntersection(env, NUM_LANES, SERVICE_RATE)
WAIT_TIMES = []

In [ ]:
env.process(vehicle_generator(env, intersection, ARRIVAL_RATE))
env.run(until=SIM_TIME)

In [ ]:
# Performance Metrics
avg_wait_time = np.mean(WAIT_TIMES)
queue_length = len(WAIT_TIMES) / SIM_TIME  # Approximate queue length
system_utilization = (ARRIVAL_RATE / (NUM_LANES * SERVICE_RATE))

In [ ]:
print(f"Average Wait Time: {avg_wait_time:.2f} minutes")
print(f"Approximate Queue Length: {queue_length:.2f} vehicles")
print(f"System Utilization: {system_utilization:.2f}")